# Test

In [1]:
import praw

# 初始化 Reddit 實例
reddit = praw.Reddit(
    client_id="Adgs1BqKFWPstMUG3G4MZA",        # 你從 Reddit 應用程序獲取的 client_id
    client_secret="G5ekLZQxg43T87UTrmwxZcacCGkGeQ",# 你從 Reddit 應用程序獲取的 client_secret
    user_agent="Chatbot",      # 一個描述你的應用程序用途的字符串
)

# 測試：打印某個 subreddit 的前 10 條熱門帖子
subreddit = reddit.subreddit("learnpython")
for submission in subreddit.hot(limit=10):
    print(submission.title)


Ask Anything Monday - Weekly Thread
Module Six Milestone IT-140
I decided to start learning Python, but have a question.
Type-hinting Overloaded Operators
What's the difference between the 'as' keyword and thewalrus operator (:=)?
Python Institute 2 Exam
i can't understand how the solution of the tower of Hanoi works (recursion)
This code is running but it's not functioning properly, can anyone help?
is it possible to implement a class like this?
Lambda Polars Binary Error


1. Data Collection (25 marks)
### 1.1 Collecting Data Using Reddit API (PRAW)
We will use the PRAW library to collect question-answer pairs from Reddit. This data will be used to train our chatbot.

In [13]:
import praw
import json


# 選擇一個 subreddit
subreddit = reddit.subreddit("AskReddit")

# 使用集合來追踪已收集的問題
collected_questions = set()
qa_pairs = []

# 收集問答對
for submission in subreddit.hot(limit=2000):  # 調整 limit 來收集更多數據
    if not submission.stickied:  # 過濾置頂貼文
        question = submission.title.strip().lower()  # 去除多餘空格並轉換為小寫以進行去重
        if question not in collected_questions and len(question) > 10:  # 過濾掉過短的問題
            submission.comments.replace_more(limit=0)
            for comment in submission.comments:
                if comment.body and not comment.stickied and len(comment.body) > 20:  # 過濾短回答
                    qa_pairs.append((question, comment.body))
                    collected_questions.add(question)  # 添加到已收集的問題集合中
                    break  # 只收集每個問題的一個回答

# 將數據保存為 JSON 格式
with open('reddit_qa_data.json', 'w') as f:
    json.dump(qa_pairs, f)


### 1.2 Annotating the Dataset with Sentiment Labels
We will use a pretrained model from Hugging Face to automatically annotate the data with sentiment labels (e.g., positive, neutral, negative). These labels will later be used to influence the chatbot's responses.

In [14]:
from transformers import pipeline

# Load a pretrained sentiment analysis model
classifier = pipeline('sentiment-analysis')

# Annotate each answer with a sentiment label
annotated_data = []
for question, answer in qa_pairs:
    sentiment = classifier(answer)[0]
    annotated_data.append({'question': question, 'answer': answer, 'label': sentiment['label']})

# Save the annotated data
with open('annotated_reddit_qa_data.json', 'w') as f:
    json.dump(annotated_data, f)


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


# 2. Preprocessing (20 marks)
### 2.1 Basic Text Preprocessing
We will preprocess the text by tokenizing, removing stop words, lemmatizing, and converting to lowercase.

In [15]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Tokenize
    tokens = word_tokenize(text)
    # Remove stop words
    filtered_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    # Lemmatize
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    return ' '.join(lemmatized_tokens)

# Preprocess all texts in the dataset
for item in annotated_data:
    item['preprocessed_question'] = preprocess_text(item['question'])
    item['preprocessed_answer'] = preprocess_text(item['answer'])


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/lichengwang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lichengwang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/lichengwang/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### 2.2 Handling Special Text Challenges
We will handle special characters, emojis, hashtags, etc., to ensure the data is clean and suitable for training.

In [17]:
import emoji

def preprocess_special_tokens(text):
    # Remove emojis
    text = emoji.replace_emoji(text, replace='')
    # Remove hashtags
    text = text.replace('#', '')
    return text

# Apply special token preprocessing
for item in annotated_data:
    item['preprocessed_question'] = preprocess_special_tokens(item['preprocessed_question'])
    item['preprocessed_answer'] = preprocess_special_tokens(item['preprocessed_answer'])


# 3. Feature Extraction (20 marks)
We will use a pretrained BERT model to extract semantic embeddings from the text. These embeddings will be used as input to the Seq2Seq model.

In [18]:
import numpy as np
import torch
from transformers import BertTokenizer, BertModel

# Load the pretrained BERT model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

def get_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', max_length=512, truncation=True, padding=True)
    with torch.no_grad():  # No need to compute gradients
        outputs = bert_model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy()

# Generate BERT embeddings for the questions
bert_embeddings = np.array([get_bert_embeddings(item['preprocessed_question']) for item in annotated_data])


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

# 4. Model Selection and Training (15 marks)
### 4.1 Building the Seq2Seq Model
We will build and train a Seq2Seq model using the BERT embeddings as input to the encoder.

In [ ]:
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model

# Get the dimension of BERT embeddings
embedding_dim = bert_embeddings.shape[2]

# Define the Seq2Seq model
encoder_inputs = Input(shape=(embedding_dim,))  # BERT embeddings as input
encoder_lstm = LSTM(units=512, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

decoder_inputs = Input(shape=(None,))  # Decoder input remains as sequence data
decoder_embedding = Dense(embedding_dim, activation='relu')(decoder_inputs)  # Embedding layer
decoder_lstm = LSTM(units=512, return_sequences=True, return_state=False)
decoder_outputs = decoder_lstm(decoder_embedding, initial_state=[state_h, state_c])

decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Prepare decoder input data
decoder_input_data = np.array([get_bert_embeddings(item['preprocessed_answer']) for item in annotated_data])

# Train the model
model.fit([bert_embeddings, decoder_input_data], target_sequences, epochs=10, batch_size=64)


### 4.2 Model Evaluation
We will evaluate the model on a test set and interpret the results.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(bert_embeddings, target_sequences, test_size=0.2, random_state=42)

# Train the model and make predictions
model.fit([X_train, y_train], y_train, epochs=10, batch_size=64)
y_pred = model.predict([X_test, y_test])

# Evaluate model performance
accuracy = accuracy_score(y_test, y_pred.argmax(axis=-1))
print(f"Model Accuracy: {accuracy}")
print(classification_report(y_test, y_pred.argmax(axis=-1)))


# 5. Deployment and Interface Development (10 marks)
### 5.1 Developing a Simple Chatbot Interface
We will use Tkinter to create a simple interface that allows users to input questions and get responses generated by the model in real-time.

In [ ]:
import tkinter as tk

def generate_response():
    user_input = entry.get()
    preprocessed_input = preprocess_text(user_input)
    input_bert_embedding = get_bert_embeddings(preprocessed_input)
    
    prediction = model.predict([input_bert_embedding, decoder_input_data])
    predicted_seq = prediction.argmax(axis=-1)
    response = tokenizer.sequences_to_texts(predicted_seq)[0]
    
    result_label.config(text=response)

# Simple Tkinter UI
root = tk.Tk()
root.title("Seq2Seq Chatbot")

entry = tk.Entry(root, width=50)
entry.pack()

button = tk.Button(root, text="Generate Response", command=generate_response)
button.pack()

result_label = tk.Label(root, text="")
result_label.pack()

root.mainloop()
